In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import io
import gzip
import astropy.io.fits as fits

In [ ]:
xmatch_spicy = pd.read_parquet('/media3/CRP7/hosts/data/SPICY/SPICY_CROSSMATCHED_2_ASEC_SMALL/')

In [ ]:
xmatch_spicy.columns

In [ ]:
# number of stamps set to plot
n_to_plot = 30

# select a random alert
random_object_indices = np.random.choice(np.arange(xmatch_spicy.shape[0]), size=n_to_plot, replace=False)

In [ ]:
def read_spicy_fits_hdu(xmatch_spicy, column, object_indx, get_last_stamp):
    if get_last_stamp:
        stamp_id = -1
    else:
        stamp_id = 0
    bytes_str = xmatch_spicy[column].values[object_indx][stamp_id]
    hdu_list = fits.open(gzip.open(io.BytesIO(bytes_str)))
    primary_hdu = hdu_list[0]
    return primary_hdu

def plot_object_cutouts(xmatch_spicy, object_indices, get_last_stamp=False):
    columns = [
        'b:cutoutScience_stampData_small',
        'b:cutoutTemplate_stampData_small',
        'b:cutoutDifference_stampData_small'
    ]
    title = ['Science', 'Template', 'Difference']

    for indx in object_indices:
        fig, axes = plt.subplots(1,3, figsize=(15,5))
        objid = xmatch_spicy.iloc[indx]['objectId']
        fwhm = str(round(xmatch_spicy.iloc[indx]['i:fwhm'][0], 3))
        aimage = str(round(xmatch_spicy.iloc[indx]['i:aimage'][0], 3))
        bimage = str(round(xmatch_spicy.iloc[indx]['i:bimage'][0], 3))
        isdiffpos = str(bool(xmatch_spicy.iloc[indx]['i:isdiffpos'][0]))
        classname = str(xmatch_spicy.iloc[indx]['v:classification'][0])
        
        fig.suptitle('indx: ' + str(indx) + '  -  ' + objid + ' ; ' + classname + '\n  fwhm = ' + fwhm + ', ' + \
                    'aimage: ' + aimage + ',  bimage: ' + bimage + ', isdiffpos: ' + isdiffpos)
    
        for i,ax in enumerate(axes.flat):
            ax.set_title(title[i])

            hdu = read_spicy_fits_hdu(xmatch_spicy, columns[i], indx, get_last_stamp)
            ax.imshow(hdu.data.astype(float))
    
        plt.show()


In [ ]:
plot_object_cutouts(xmatch_spicy, random_object_indices, get_last_stamp=True)

In [ ]:
plot_object_cutouts(xmatch_spicy, random_object_indices, get_last_stamp=False)